In [ ]:
import ee
import geemap
import os
import geopandas as gpd
ee.Authenticate()

import pandas as pd
from datetime import datetime
from datetime import timedelta
import time, math
import logging

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.ERROR)  # 设置记录级别为 ERROR 或更高级别

file_handler = logging.FileHandler('warnings.log')
logger.addHandler(file_handler)

def fmask(image):
    cloudsBitMask = (1 << 3)
    cloudshadowBitMask = (1 << 4)

    qaMask = image.select('QA_PIXEL').bitwiseAnd(cloudsBitMask).eq(0) \
                        .And(image.select('QA_PIXEL').bitwiseAnd(cloudshadowBitMask).eq(0))
    saturationMask = image.select('QA_RADSAT').eq(0)

    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0).add(-273.15);
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True).updateMask(qaMask).updateMask(saturationMask)

def cal_ice_snow(image):
    blue = image.select('SR_B1') 
    green = image.select('SR_B2')
    ice_snow = blue.add(green).gt(0.45)
    snowBitMask = (1 << 5)
    qaMask = image.select('QA_PIXEL').bitwiseAnd(snowBitMask).eq(0)
    snowMask = qaMask.lt(1)
    ice_snow2 = ice_snow.gt(0).Or(snowMask.gt(0)).rename('j8')

    cal_ice_snow = ice_snow2.reduceRegion(
                        reducer = ee.Reducer.mean(),
                        geometry = geometry,
                        scale = 30,
                        maxPixels = 1e13,
                    )
    return image.set('ice_snow_coverage',cal_ice_snow.get('j8'))

ls5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
ls7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
merged = ls5.merge(ls7)


result = pd.DataFrame(columns = ["lat_wgs84", "lon_wgs84", "obs_date", "obs_time", "site_id",  "site_country",
                                  "param_code", "param_name", "obs_value", "unit", "source",'image_id','imgae_time',
                                'time_diff','Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2','temp','prec','wind','solar',
                                 'evaporation','lai_h','lai_l','LST'])

index_l57 = {
    'ndwi': lambda image: image.normalizedDifference(['SR_B2', 'SR_B4']).rename('ndwi'),
    'mndwi': lambda image: image.normalizedDifference(['SR_B2', 'SR_B5']).rename('mndwi'),
    'AWEIsh': lambda image: image.expression(
        'BLUE + 2.5 * GREEN - 1.5 * (NIR + SWIR1) - 0.25 * SWIR2', {
            'BLUE': image.select('SR_B1'),
            'GREEN': image.select('SR_B2'),
            'NIR': image.select('SR_B4'),
            'SWIR1': image.select('SR_B5'),
            'SWIR2': image.select('SR_B7')
        }).float().rename('AWEIsh')}

for index,row in df.iloc[577000:700000].iterrows():
    if index % 500 ==0:
        print(index)

    x = row['lon_wgs84']
    y = row['lat_wgs84']
    date = row['obs_date']

    start_date = date - timedelta(days=3)
    end_date =  date + timedelta(days=3)

    roi = ee.Geometry.Point([x, y])
    geometry = roi.buffer(45)
    # Map.centerObject(roi,15)

    # Map.addLayer(roi, {'color': 'green'}, 'roi')
    # Map.addLayer(geometry, {'color': 'red'}, 'buffer')

    image = (merged.filterBounds(geometry) \
            .filterDate(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')) \
            .map(fmask) \
            .map(cal_ice_snow) \
            .filter(ee.Filter.lt('ice_snow_coverage',0.01))
            .first())
    try:
        band_list = image.bandNames().getInfo()
    except:
        continue

    mndwi_method = index_l57['mndwi'](image).gt(0)
    AWEIsh_method = index_l57['AWEIsh'](image).gt(0)

    image_final = image.updateMask(AWEIsh_method).updateMask(mndwi_method).clip(geometry)

    vis_params = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'], 'min': 0.0, 'max': 0.2}
    # Map.addLayer(image, vis_params, 'water_image')

    pixel_value = image_final.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']).reduceRegion(
                reducer = ee.Reducer.median(),
                geometry = geometry,
                scale = 30,
                maxPixels = 1e9,
            ).getInfo()

    if pixel_value['SR_B1'] is None:
      continue
    else:
      median = [round(value, 4) for value in pixel_value.values()]

    date_ls = ee.Date(image.get('system:time_start').getInfo())
    formatted_date = date_ls.format('yyyy-MM-dd HH:mm:ss').getInfo()
    try:
      insitu_date = datetime.combine(row['obs_date'].date(), row['obs_time']).strftime('%Y-%m-%d %H:%M:%S')
      time_diff = round((pd.to_datetime(formatted_date) - pd.to_datetime(insitu_date)).total_seconds() / 3600,2)

    except:
      continue
    image_id = image.get('system:index').getInfo()


    era5 = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
        .filterBounds(geometry) \
        .filter(ee.Filter.date(date_ls.advance(-1, 'day'), date_ls.advance(1, 'day'))) \
        .first().clip(geometry).reduceRegion(reducer=ee.Reducer.mean(),geometry=geometry,scale=30).getInfo()
    try:
      temp = era5.get('temperature_2m')-273.15
      prec = era5.get('total_precipitation_sum')
      wind_u = era5.get('u_component_of_wind_10m')
      wind_v = era5.get('v_component_of_wind_10m')
      wind = math.sqrt(wind_u**2+wind_v**2)
      solar = era5.get('surface_net_solar_radiation_sum')
      evaporation = era5.get('evaporation_from_open_water_surfaces_excluding_oceans_sum')
      lai_h = era5.get('leaf_area_index_high_vegetation')
      lai_l = era5.get('leaf_area_index_low_vegetation')

      lst = image_final.select('ST_B6').reduceRegion(
                  reducer = ee.Reducer.median(),
                  geometry = geometry,
                  scale = 30,
                  maxPixels = 1e9,
              ).getInfo()

      row_list = []

      row_list = [row["lat_wgs84"],row["lon_wgs84"],row["obs_date"].strftime('%Y-%m-%d'),row["obs_time"].strftime('%H:%M:%S'),
                  row["site_id"],row["site_country"],row["param_code"],row["param_name"],row["obs_value"],row["unit"],row["source"]] \
      +[image_id,formatted_date,abs(time_diff)]+median+[round(temp,2),prec,round(wind,2),int(solar),round(evaporation,8),round(lai_h,2),round(lai_l,2),round(lst['ST_B6'],2)]
    except:
      continue

    result.loc[len(result)] = pd.Series(row_list, index=result.columns)
    print(row_list)